In [ ]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import matplotlib.pyplot as plt


In [ ]:
transforms=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
train_data=torchvision.datasets.CIFAR100(root='./data',train=True,download=True,transform=transforms)
test_data=torchvision.datasets.CIFAR100(root='./data',train=False,download=True,transform=transforms)
train_loader=DataLoader(train_data,batch_size=32,shuffle=True, num_workers=2)
test_loader=DataLoader(test_data,batch_size=32,shuffle=True, num_workers=2)

In [ ]:
train_loader=DataLoader(train_data,batch_size=32,shuffle=True, num_workers=2)
test_loader=DataLoader(test_data,batch_size=32,shuffle=True, num_workers=2)

In [ ]:
image,label=train_data[0]
print(image.shape)
print(label)

In [ ]:
class_names = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl',
    'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee',
    'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish',
    'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange',
    'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk',
    'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe',
    'whale', 'willow_tree', 'wolf', 'woman', 'worm'
]

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1=nn.Conv2d(3,12,5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(12,16,5)
    self.fc1=nn.Linear(16*5*5,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,100)
  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=torch.flatten(x,1)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x

In [ ]:

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1=nn.Conv2d(3,12,5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(12,16,5)
    self.fc1=nn.Linear(16*5*5,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,100)
  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=torch.flatten(x,1)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x
best_loss = float('inf')
patience = 5
counter = 0
best_model_params = None

# List to store losses for plotting
losses = []

for epoch in range(300):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    losses.append(avg_loss)

    print(f'[{epoch+1},{i+1:5d}] loss: {avg_loss:.3f}')

    if avg_loss < best_loss:
        best_loss = avg_loss
        counter = 0
        best_model_params = net.state_dict().copy()
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping triggered at epoch {epoch+1}')
            net.load_state_dict(best_model_params)
            break

plt.figure(figsize=(10, 5))
plt.plot(range(1, len(losses) + 1), losses)
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
best_model_params

In [ ]:
torch.save(net.state_dict(),'cnnet_224epoch.pth')

In [ ]:
correct=0
total=0
net.eval()
with torch.no_grad():
  for data in test_loader:
    images,labels=data
    outputs=net(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()

acc=100*correct/total
print(f'Accuracy : {acc}%')

In [ ]:
class tinyNet(nn.Module):
  def __init__(self):
    super(tinyNet,self).__init__()
    self.conv1=nn.Conv2d(3,12,5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(12,16,5)
    self.fc1=nn.Linear(16*5*5,1024)
    self.fc2=nn.Linear(1024,512)
    self.fc3=nn.Linear(512,256)
    self.fc4=nn.Linear(256,100)
  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=torch.flatten(x,1)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x

In [ ]:
net2=tinyNet()
loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(net2.parameters(),lr=0.001,momentum=0.9)

In [ ]:


best_loss = float('inf')
patience = 3
counter = 0
best_model_params = None

losses = []

for epoch in range(300):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net2(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    losses.append(avg_loss)

    print(f'[{epoch+1},{i+1:5d}] loss: {avg_loss:.3f}')

    if avg_loss < best_loss:
        best_loss = avg_loss
        counter = 0
        best_model_params = net2.state_dict().copy()
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping triggered at epoch {epoch+1}')
            net2.load_state_dict(best_model_params)
            break

plt.figure(figsize=(10, 5))
plt.plot(range(1, len(losses) + 1), losses)
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
torch.save(net2.state_dict(),'tinycnnet_62epoch.pth')

In [ ]:
correct=0
total=0
net2.eval()
with torch.no_grad():
  for data in test_loader:
    images,labels=data
    outputs=net2(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()

acc=100*correct/total
print(f'Accuracy : {acc}%')

In [ ]:
import torch
import torch.nn as nn
from torch.nn import (Conv2d, Linear, MaxPool2d, Module, ReLU, Sequential, Softmax)
from typing import Callable, Optional

ActivT = Optional[Callable[[], Module]]

def make_conv_pool_activ(
    in_channels: int,
    out_channels: int,
    kernel_size: int,
    activation: ActivT = None,
    pool_size: Optional[int] = None,
    pool_stride: Optional[int] = None,
    **conv_kwargs
):
    layers = [Conv2d(in_channels, out_channels, kernel_size, **conv_kwargs)]
    if activation:
        layers.append(activation())
    if pool_size is not None:
        layers.append(MaxPool2d(pool_size, stride=pool_stride))
    return layers

class Classifier(Module):
    def __init__(
        self, convs: Sequential, linears: Sequential, use_softmax: bool = True
    ):
        super().__init__()
        self.convs = convs
        self.linears = linears
        self.softmax = Softmax(1) if use_softmax else Sequential()

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        outputs = self.convs(inputs)
        return self.softmax(self.linears(outputs.view(outputs.shape[0], -1)))

class BasicBlock(Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = Sequential(
                Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = ReLU()(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = ReLU()(out)
        return out

class ResNet18(Classifier):
    def __init__(self, num_classes=100): 
        self.in_planes = 64

        def _make_layer(block, planes, num_blocks, stride):
            strides = [stride] + [1]*(num_blocks-1)
            layers = []
            for stride in strides:
                layers.append(block(self.in_planes, planes, stride))
                self.in_planes = planes * block.expansion
            return Sequential(*layers)

        convs = Sequential(
            *make_conv_pool_activ(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            ReLU(),
            MaxPool2d(kernel_size=3, stride=2, padding=1),
            _make_layer(BasicBlock, 64, 2, 1),
            _make_layer(BasicBlock, 128, 2, 2),
            _make_layer(BasicBlock, 256, 2, 2),
            _make_layer(BasicBlock, 512, 2, 2),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        linears = Sequential(Linear(512 * BasicBlock.expansion, num_classes))
        super().__init__(convs, linears)

def model(file=None):
    net = ResNet18().float()
    if file is not None:
        net.load_state_dict(torch.load(file))
    return net

In [ ]:
train_data=torchvision.datasets.CIFAR100(root='./data',train=True,download=True,transform=transforms)
test_data=torchvision.datasets.CIFAR100(root='./data',train=False,download=True,transform=transforms)
train_loader=DataLoader(train_data,batch_size=32,shuffle=True, num_workers=2)
test_loader=DataLoader(test_data,batch_size=32,shuffle=True, num_workers=2)

In [ ]:
resnet = ResNet18().float()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)